<a href="https://colab.research.google.com/github/tosiki1202/GenAI-app/blob/main/20250924.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install transformers

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 使用したいモデルの名前を指定
model_name = "rinna/japanese-gpt2-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [9]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=32000, bias=False)
)

In [11]:
# 入力テキストの設定
input_text = "良い天気ですね"

# トークナイズとテンソル変換
inputs = tokenizer(input_text, return_tensors="pt").to(device)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# テキスト生成
output = model.generate(
    input_ids,
    attention_mask=attention_mask, # attention_maskを明示的に渡す
    max_length=100,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

良い天気ですね今日も晴れますように。 明日もまた雨の予報です 今日からまた仕事ですのでお気をつけてお仕事お休みください


# Task
Webスクレイピングと要約処理を実装し、指定されたURLのWebページの内容を要約してください。

## Webスクレイピングライブラリのインストール

### Subtask:
Webスクレイピングに必要なライブラリをインストールします。


**Reasoning**:
Install the required libraries for web scraping using pip.



In [12]:
!pip install beautifulsoup4 requests

## Webページの取得

### Subtask:
指定されたURLのWebページの内容を取得します。


**Reasoning**:
Import the requests library and get the content of the specified URL. Check if the request was successful and store the content or print an error message.



In [13]:
import requests

# 取得したいWebページのURLを定義
url = "https://www.nikkei.com/"

# requests.get()関数を使用して、指定したURLからWebページの内容を取得
response = requests.get(url)

# 取得したレスポンスが成功したかどうか（ステータスコードが200番台であるか）を確認
if response.status_code >= 200 and response.status_code < 300:
    # 成功した場合、レスポンスのテキストコンテンツを文字列として変数に格納
    html_content = response.text
    print("Webページの取得に成功しました。")
    # print(html_content[:500]) # 確認のため、一部を表示
else:
    # 失敗した場合、エラーメッセージを表示
    print(f"Webページの取得に失敗しました。ステータスコード: {response.status_code}")


Webページの取得に成功しました。


## テキスト抽出

### Subtask:
取得したWebページから要約対象となるテキストを抽出します。


**Reasoning**:
Extract text content from the HTML using BeautifulSoup.



In [14]:
from bs4 import BeautifulSoup

# BeautifulSoupオブジェクトを作成
soup = BeautifulSoup(html_content, 'html.parser')

# 要約対象となるタグ（p, h1-h6）をすべて見つける
tags = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

# 抽出したテキストコンテンツを結合
extracted_text = ""
for tag in tags:
    extracted_text += tag.get_text() + "\n"

# 抽出されたテキストの一部を表示して確認
print("テキストの抽出に成功しました。")
print(extracted_text[:500])

テキストの抽出に成功しました。
日本経済新聞
NIKKEI Primeについて
朝夕刊や電子版ではお伝えしきれない情報をお届けします。今後も様々な切り口でサービスを開始予定です。
米中談合、対立より悪夢トランプ氏が好む縄張り支配
■派手なディールを熱望するトランプ氏
■その執着心を逆手にとってじらす中国
■世界は米中の談合による混沌に備えを…
出遅れ「デジタル教科書」、英語や理科に効果期待海外では見直しも
中央教育審議会の作業部会は24日、デジタル教科書を正式な教科書として認めるとした最終まとめを了承した。海外では活用を広げる国がある一方、紙に回帰する動きもある。明治の学校制度の開始以来、紙だった教科書が変わることで、学力向上につながるのか。教員の力量が問われることになる。

「教科書での学びの可能性を大きく広げようとするものだ」。最終まとめは冒頭で、デジタル教科書の正式化の意義をこう強調した。次期…
トランプ氏「ウクライナは全領土奪還できる」割譲要求から一転、ロシアに圧力
【ワシントン=坂口幸裕、ニューヨーク=北松円香】トランプ米大統領は23日、欧州の支援があればウクライナがロシアから全土を奪還できると表明した。和


## テキスト要約モデルの準備

### Subtask:
テキスト要約に使用するモデルとトークナイザーをロードします。


**Reasoning**:
Load the model and tokenizer for text summarization.



**Reasoning**:
Tokenize the extracted text, generate the summary using the loaded model, decode the output, and store it in a variable.



In [16]:
import torch

# モデルを適切なデバイスに移動
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# テキストが長すぎる場合、モデルの最大入力長（このモデルは1024トークン）に合わせて調整
max_model_input_length = tokenizer.model_max_length
if len(extracted_text) > max_model_input_length:
    # 最初の部分を使用
    extracted_text = extracted_text[:max_model_input_length]

# 抽出したテキストをトークナイズ
inputs = tokenizer(extracted_text, return_tensors="pt", truncation=True, max_length=max_model_input_length).to(device)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# テキスト生成（要約）
# 以前のテキスト生成で使用したパラメータを参考に設定
summary_output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=200,  # 生成する要約の最大長を調整
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

# 生成された要約をデコード
generated_summary = tokenizer.decode(summary_output[0], skip_special_tokens=True)

print("要約の生成に成功しました。")
print(generated_summary)

OverflowError: int too big to convert

In [21]:
print("--- 生成された要約結果 ---")
print(generated_summary)

--- 生成された要約結果 ---
日本経済新聞  rimeについて 朝夕刊や電子版ではお伝えしきれない情報をお届けします。今後も様々な切り口でサービスを開始予定です。 米中談合、対立より悪夢トランプ氏が好む縄張り支配 ■派手なディールを熱望するトランプ氏 ■その執着心を逆手にとってじらす中国 ■世界は米中の談合による混沌に備えを... 出遅れ「デジタル教科書」、英語や理科に効果期待海外では見直しも 中央教育審議会の作業部会は24日、デジタル教科書を正式な教科書として認めるとした最終まとめを了承した。海外では活用を広げる国がある一方、紙に回帰する動きもある。明治の学校制度の開始以来、紙だった教科書が変わることで、学力向上につながるのか。教員の力量が問われることになる。 「教科書での学びの可能性を大きく広げようとするものだ」。最終まとめは冒頭で、デジタル教科書の正式化の意義をこう強調した。次期... トランプ氏「ウクライナは全領土奪還できる」割譲要求から一転、ロシアに圧力 【ワシントン=坂口幸裕、ニューヨーク=北松円香】トランプ米大統領は23日、欧州の支援があればウクライナがロシアから全土を奪還できると表明した。和平合意へウクライナに一部の米軍基地が移設される可能性も排除できないという。...
